In [1]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset

In [2]:
ds = load_dataset("jlbaker361/wikiart")

In [ ]:
# constants
HEIGHT = 64
WIDTH = 64
CHANNELS = 3
BATCH_SIZE = 32

# Image Size Normalization, Augumentation


In [7]:

train_ds = ds['train'].select(range(5000))
def convert_img(x):
    import numpy as np  #subprocesses lmao
    HEIGHT = 64
    WIDTH = 64
    x['img_pixels'] = np.array(x['image'].resize((WIDTH, HEIGHT))).reshape(-1)/255
    return x

train_ds = train_ds.map(convert_img, num_proc=4)
train_ds = train_ds.filter(lambda x: len(x['img_pixels']) == HEIGHT*WIDTH*3)

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [16]:
def augment_with_tf(image):
    image = tf.reshape(image, [HEIGHT, WIDTH, CHANNELS])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    return image


Map:   0%|          | 0/4989 [00:00<?, ? examples/s]

In [ ]:
def preprocess_and_augment(examples):
    examples['img_pixels'] = [augment_with_tf(img) for img in examples['img_pixels']]
    return examples

In [ ]:
train_ds_augmented = train_ds.map(preprocess_and_augment, batched=True)

In [ ]:
def preprocess_batch(ds, batch_size=BATCH_SIZE):
    def generator():
        for row in ds:
            yield row['img_pixels'], row['label']
    dataset = tf.data.Dataset.from_generator(
        generator, 
        output_signature=(
            tf.TensorSpec(shape=(HEIGHT, WIDTH, CHANNELS), dtype=tf.float32),  # Image shape
            tf.TensorSpec(shape=(), dtype=tf.int64)  # Label shape
        )
    )
    dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)
    return dataset

In [ ]:
train_dataset = preprocess_batch(train_ds_augmented)

In [17]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>